**Javier Rojas Herrera**

jrojash1995@gmail.com

**Hardware, Deployment y MLOps**



### Letra pequeña del Deep Learning

* **Entrenamiento e Inferencia requieren de hardware avanzado en computo numérico**
* Se requieren grandes volumenes de datos etiquetados



###  <center>[<img src="images/gpuvscpuvstpu.webp" width="80%"/> ](attachment:image.png)</center>

## GPU (graphics processing unit)
###  <center>[<img src="images/A100.jpg" width="60%"/> ](attachment:image.png)</center>

## GPU vs CPU
###  <center>[<img src="images/gpuvscpu.png" width="60%"/> ](attachment:image.png)</center>

## GPU vs CPU: Inferencia
###  <center>[<img src="images/gpuvscpu2.png" width="70%"/> ](attachment:image.png)</center>

## Tabla resumen GPUS

|GPU |  Cuda cores | Tensor cores | VRAM  | Power | Precio |
|----------|----------|----------| ----------|  ----------| ----------|
| T4    | 2500   | 320  | 15 GB | 70 W | 1100 usd |
| L4   | 7680   | 240  | 24 GB | 72 W | 2600 usd |
| L40   | 18176    | 568  | 48 GB | 300 W | 8400 usd |
| A100    | 6920   |  422   | 80 GB | 400 W | 12000 usd |
| H100    | 14592    |  456    | 80 GB | 350 W | 30000 usd |


In [1]:
!nvidia-smi

Tue Aug 20 09:44:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   35C    P8              20W / 320W |    224MiB / 10240MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from torchvision.models import vit_b_16 , ViT_B_16_Weights
from torchvision.transforms import v2
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader,Subset
import torch
import matplotlib.pyplot as plt
import time
from trans import UnNormalize
from io import StringIO
import numpy as np

In [3]:
##load model VIT B 16
weights = ViT_B_16_Weights.DEFAULT
preprocess = weights.transforms()
model = vit_b_16(weights=weights)
model.heads.head = torch.nn.Linear(768,10)

##set optimizer and loss
optim = torch.optim.Adam(model.parameters())
cross_entropy = torch.nn.CrossEntropyLoss()

##load data into dataloader
data = CIFAR10("./", download=True, train=True, transform=weights.transforms())
data = CIFAR10("./", download=True, train=True, transform=weights.transforms())
subset_indices = torch.randperm(len(data))[:1000]
subset_cifar10 = Subset(data, subset_indices)
dataloader = DataLoader(subset_cifar10, batch_size=32, shuffle=False)

class_names = ["Airplane","Auto","Bird","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]

Files already downloaded and verified
Files already downloaded and verified


In [4]:
std = weights.transforms().std
mean = weights.transforms().mean
invTrans=UnNormalize(mean,std)
@interact
def show_articles_more_than(x=1000):
    plt.figure(figsize=(5,3))
    print("Label: ",class_names[data[x][1]])
    plt.axis('off')
    plt.imshow(invTrans(data[x][0]).permute(1,2,0))

interactive(children=(IntSlider(value=1000, description='x', max=3000, min=-1000), Output()), _dom_classes=('w…

# ¿Qué elementos utilizan VRAM en un entrenamiento?

---

- Almacenamiento de tensores de entrada


- Almacenamiento de los parametros del modelo (weight and biases)

- Almacenamiento de gradientes (backpropagation)

- Almacenamiento de tensores de salida

---

In [5]:
## Tomando un batch del dataloader y transfiriendolo a VRAM
for image,label in dataloader:
    image_=image.cuda()
    break
!nvidia-smi

Tue Aug 20 09:44:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   36C    P2              27W / 320W |    463MiB / 10240MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
## transfiriendo el modelo a VRAM
model = model.cuda()
!nvidia-smi

Tue Aug 20 09:44:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.29.06              Driver Version: 545.29.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080        Off | 00000000:07:00.0 Off |                  N/A |
|  0%   38C    P2              59W / 320W |    845MiB / 10240MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Locura de los parámetros


# <center>[ <img src="images/madness.png" width="60%"/>](attachment:image.png)</center>

## ¿Cómo aprovechar al maximo el hardware disponible?

### ¿Cómo los computadores almacenan los numeros reales?


* Estandar IEEE754 establece la forma en la que los numeros reales son almacenados en memoria

* Existen los puntos flotante de 16,32,64,128 bits. Siendo el más utilizado el punto flotante de 32 bits o de precisión simple

# FP32 vs FP16

 # <center>[ <img src="images/fp16.ppm" width="80%"/>](attachment:image.png)</center>


### FP32 expresión notación científica
${(−1)^S × 2^{(E-127)} × 1.F}$   

Donde:

$S = signo$

$E = exponente$

$F = mantisa$

### FP16 expresión notación científica
${(−1)^S × 2^{(E-15)} × 1.F}$   

Donde:

$S = signo$

$E = exponente$

$F = mantisa$

In [7]:
number= 0.123456789123456789123
sio = StringIO()
np.savetxt(sio, np.array([number], dtype=np.float64))
np.savetxt(sio, np.array([number], dtype=np.float32))
np.savetxt(sio, np.array([number], dtype=np.float16))
s = sio.getvalue()
print(s)


1.234567891234567838e-01
1.234567910432815552e-01
1.234741210937500000e-01



# FP32 vs FP16

 # <center>[ <img src="images/fp32vsfp16_tabla.png" width="80%"/>](attachment:image.png)</center>

In [22]:
A= np.random.rand(5000,5000).astype(np.float32)
B= np.random.rand(5000,5000).astype(np.float32)

In [23]:
%%time
print(np.matmul(A,B))

[[1252.1611 1225.4093 1235.9021 ... 1236.6078 1240.352  1248.9011]
 [1253.1814 1260.9763 1243.3846 ... 1252.232  1248.1768 1273.7944]
 [1234.982  1225.7998 1223.8563 ... 1234.9598 1223.2595 1242.1187]
 ...
 [1264.1403 1249.7083 1258.812  ... 1251.8722 1247.9136 1274.4338]
 [1243.8434 1240.2603 1227.7488 ... 1228.9077 1222.6744 1249.3503]
 [1241.6654 1239.8179 1214.8789 ... 1242.3411 1235.4119 1242.6171]]
CPU times: user 4.46 s, sys: 22.6 ms, total: 4.49 s
Wall time: 375 ms


In [24]:
A= np.random.rand(5000,5000).astype(np.float64)
B= np.random.rand(5000,5000).astype(np.float64)

In [25]:
%%time
print(np.matmul(A,B))

[[1249.69263886 1264.36179625 1236.71745103 ... 1244.55155038
  1239.56847115 1241.85963489]
 [1268.12400729 1265.97013354 1271.70727464 ... 1263.82112103
  1248.87865961 1263.65998639]
 [1224.42082935 1234.9236422  1216.74991232 ... 1213.03525978
  1216.03054201 1228.31899213]
 ...
 [1250.85675417 1253.5641412  1240.10554609 ... 1259.07204147
  1240.8052213  1252.44091154]
 [1231.70855692 1241.47769454 1235.34692057 ... 1227.44051865
  1225.86423203 1237.94136802]
 [1254.15307367 1265.07278831 1240.04607975 ... 1251.95226251
  1251.75534882 1249.40062106]]
CPU times: user 9.66 s, sys: 342 ms, total: 10 s
Wall time: 837 ms


## ¿Es posible usar representacion de FP16 para entrenar o inferir?

* Algunas operaciones como las convoluciones o lineales, pueden realizarse completamente en FP16
* Sin embargo, otras operaciones como la reducción, a menudo pueden necesitar la representacion en FP32

## Precision mixta automática (AMP)

 # <center>[ <img src="images/amp.png"/>](attachment:image.png)</center>

## ¿Realmente tiene beneficios usar FP16?

  # Tensor cores
 
 # <center>[ <img src="images/tensorop.png" width=100%/>](attachment:image.png)</center>

 # Tensor cores
 
 # <center>[ <img src="images/tensor_cores.gif"/>](attachment:image.png)</center>

### Entrenamiento tradicional

In [26]:
for epoch in range(1):
    time_i=time.time()
    epoch_loss = 0.0
    for image,label in dataloader:
        optim.zero_grad() 
        image=image.cuda()  
        label=label.cuda()
        output = model(image)   
        loss= cross_entropy(output,label)        
        loss.backward()     
        epoch_loss+=loss.item()
        optim.step()
    print(f'Tiempo por epoca: {time.time()-time_i} segs | Epoch loss: {epoch_loss}')        

Tiempo por epoca: 9.394094944000244 segs | Epoch loss: 80.61469578742981


## Entrenamiento utilizando precision mixta + tensor cores

In [27]:
for epoch in range(1):
    time_i=time.time()
    epoch_loss = 0.0
    for image,label in dataloader:
        optim.zero_grad()
        image=image.cuda()
        label=label.cuda()
        with torch.autocast(device_type="cuda"):
            output = model(image)
            loss= cross_entropy(output,label)        
            loss.backward()
            optim.step()
        epoch_loss+=loss.item()
    print(f'Tiempo por epoca: {time.time()-time_i} segs | Epoch loss: {epoch_loss}')

Tiempo por epoca: 4.504239559173584 segs | Epoch loss: 70.95635986328125


## ¿Qué problemas pueden ocurrir al trabajar con una precisión de 16 bits?

* Cálculo de gradientes acumulativos podrian no poder representarse en FP16 (Desvanecimiento de gradiente)

### Entrenamiento con cálculo de gradiente escalado

In [30]:
scaler =torch.amp.GradScaler("cuda") 
a=torch.tensor([0.00045],requires_grad=True).cuda()
scaler.scale(a)

tensor([29.4912], device='cuda:0', grad_fn=<MulBackward0>)

In [31]:
scaler = torch.amp.GradScaler("cuda") 
for epoch in range(3):
    time_i=time.time()
    epoch_loss = 0.0
    for image,label in dataloader:
        optim.zero_grad()
        image=image.cuda()
        label=label.cuda()
        with torch.autocast(device_type="cuda"):
            output = model(image)
            loss= cross_entropy(output,label)        
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()
        epoch_loss+=loss.item()
    print(f'Tiempo por epoca: {time.time()-time_i} segs | Epoch loss: {epoch_loss}')

Tiempo por epoca: 4.535714387893677 segs | Epoch loss: 62.873382568359375
Tiempo por epoca: 4.51518988609314 segs | Epoch loss: 61.40594482421875
Tiempo por epoca: 4.515941143035889 segs | Epoch loss: 59.82152557373047


# ¿Qué sucede si no dispongo de hardware o si requiero de pocas horas de computo?

### Principales servicios cloud para creación de máquinas virtuales

# <center>[<img src="images/azurevs.jpg" width="80%"/>](attachment:image.png)</center>

### Pros de utilizar máquinas virtuales

* Fácil de crear y configurar según las necesidades

* Costo bajo al corto plazo

* Integración directa con otros servicios cloud del mismo prestador

### Contras de utilizar máquinas virtuales

* Los recursos solicitados pueden no estar disponibles

* Alto costo a largo plazo

## ¿Usar MV es lo más eficiente para realizar tareas de machine learning en la nube?

* Los modelos en MV no escalan (Inferencia)

* Entrenamiento y despliegue de modelos complejo de automatizar

# <center>[<img src="images/mlstudiovsvertex.png" width="60%"/>](attachment:image.png)</center>

### Ventajas al utilizar servicios especializados para ML en la nube

* Deployment escalable y automatizado de modelos

* Entrenamiento automatizado (pipelines)

* Disponibilidad de una familia de modelos pre entrenados a través de API

* Creación de notebooks jupyter

# <center>[<img src="images/mlsteps.jpg" width="80%"/>](attachment:image.png)</center>

## Deployment de modelos de ML

* Disponibilizar modelos para el uso real de usuarios

# <center>[<img src="images/depl.png" width="50%"/>](attachment:image.png)</center>

## Modelo como API
# <center>[<img src="images/apimodel.png" width="70%"/>](attachment:image.png)</center>

## Frameworks para deployment de modelos

# <center>[<img src="images/deploy.png" width="70%"/>](attachment:image.png)</center>

### Ejemplo:  Bento ML

In [35]:
get_ipython().system_raw('BENTOML_PORT=11000 bentoml serve server:svc &')

2024-08-20T09:54:12-0400 [WARNING] [cli] Using lowercased runnable class name 'gpt2' for runner.
2024-08-20T09:54:12-0400 [INFO] [cli] Environ for worker 0: set CUDA_VISIBLE_DEVICES to 0
2024-08-20T09:54:12-0400 [INFO] [cli] Prometheus metrics for HTTP BentoServer from "server:svc" can be accessed at http://localhost:11000/metrics.
2024-08-20T09:54:13-0400 [INFO] [cli] Starting production HTTP BentoServer from "server:svc" listening on http://0.0.0.0:11000 (Press CTRL+C to quit)
2024-08-20T09:54:16-0400 [WARNING] [api_server:6] Using lowercased runnable class name 'gpt2' for runner.
2024-08-20T09:54:16-0400 [WARNING] [api_server:4] Using lowercased runnable class name 'gpt2' for runner.
2024-08-20T09:54:16-0400 [WARNING] [api_server:10] Using lowercased runnable class name 'gpt2' for runner.
2024-08-20T09:54:16-0400 [WARNING] [api_server:1] Using lowercased runnable class name 'gpt2' for runner.
2024-08-20T09:54:16-0400 [WARNING] [api_server:8] Using lowercased runnable class name 'gpt

/run/media/javier/miniconda3/envs/UC2/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Modelo como API en VM
# <center>[<img src="images/depl2.png" width="70%"/>](attachment:image.png)</center>

## Problemas de levantar modelos API en MV

# <center>[<img src="images/apin2.png" width="70%"/>](attachment:image.png)</center>

## Solución: Escalar modelos API en cloud

# <center>[<img src="images/depl4.png" width="50%"/>](attachment:image.png)</center>

### Mostrar ejemplo de escalamiento en VERTEX AI

## ¿Qué es MLOps?

* Paradigma repetible que tiene como objetivo implementar y mantener modelos de aprendizaje automático en producción de manera confiable y eficiente.


# <center>[<img src="images/mlops.png" width="80%"/>](attachment:image.png)</center>

## Pipelines en MLOps

* Una Pipeline es un flujo de trabajo conformado por uno o varios componentes y sus interacciones a través de entradas y salidas.

# <center>[<img src="images/compo.png" width="60%"/>](attachment:image.png)</center>

# <center>[<img src="images/pipeline.png" width="60%"/>](attachment:image.png)</center>

### Frameworks para MLOps

# <center>[<img src="images/mlops_frame2.png" width="70%"/>](attachment:image.png)</center>

### Servicios cloud para MLOps
# <center>[<img src="images/mlstudiovsvertex.png" width="60%"/>](attachment:image.png)</center>

### Ejemplo de pipeline de juguete definida en Kubeflow

In [39]:
import kfp.dsl as dsl
from kfp.v2 import compiler

@dsl.component
def add(a: float, b: float) -> float:
    return a + b

@dsl.component
def mul(a: float, b: float) -> float:
    return a * b

@dsl.pipeline
def add_pipeline(a: float, b: float):
    add_task = add(a=a, b=b)
    mul_task = mul(a=a, b=add_task.output)
    
compiler.Compiler().compile(pipeline_func=add_pipeline, package_path='add_pipeline.json')


### Ejemplo de Pipeline real en vertex AI

# <center>[<img src="images/vertex.png" width="56%"/>](attachment:image.png)</center>

### Niveles de MLOps

### Nivel 0
# <center>[<img src="images/nivel0.svg" width="80%"/>](attachment:image.png)</center>

### Nivel 1
# <center>[<img src="images/nivel1.svg" width="70%"/>](attachment:image.png)</center>

### Nivel 2
# <center>[<img src="images/nivel2.svg" width="75%"/>](attachment:image.png)</center>